# Data Cleaning

In [2]:
import pandas as pd

## Import raw datasets

In [3]:
column_name = ['total_lot','lot_type','lots_available','carpark_number','update_date','update_time']
D1 = pd.read_csv('carpark_availability_final.csv',names=column_name)
D2 = pd.read_csv('hdb-carpark-information.csv')

In [8]:
D1.head()

,total_lot,lot_type,lots_available,carpark_number,update_date,update_time
0,105,C,18,HE12,2022-09-22,20:45:44
1,583,C,288,HLM,2022-09-22,20:46:07
2,329,C,154,RHM,2022-09-22,20:45:45
3,97,C,61,BM29,2022-09-22,20:45:54
4,96,C,52,Q81,2022-09-22,20:45:56


In [9]:
D2.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.8,Y
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.1,N
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.0,N
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.0,N
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.0,N


## Resampling raw dataset to data with 15 minute time intervals

In [10]:
D1['datetime'] = D1['update_date'] + " " + D1['update_time']
D1['datetime']= pd.to_datetime(D1['datetime'])
D1['datetime'] = D1.datetime.round('15min')

## Drop duplicate data

In [11]:
D1.drop(['update_date'],axis=1,inplace=True)
D1.drop(['update_time'],axis=1,inplace=True)
D1.drop_duplicates(inplace=True)

## Drop parking lots with less than 1854 time steps

In [ ]:
for j in range(0,20):
    for i,v in D1.carpark_number.value_counts().items():
        # print(i,v)
        if v < 1854:
            D1.drop(D1[D1.carpark_number == i].index, inplace=True)

## Drop data with duplicate values based on carpark_number and datetime subset

In [ ]:
for i,v in D1.carpark_number.value_counts().items():
    print(i,v)
    D1.drop_duplicates(subset=['carpark_number','datetime'],keep='first', inplace=True)


## Save 1st cleaned version

In [14]:
D1.to_csv('data_droped_nov10.csv',sep=',')

## 2nd cleaning cycle started as found duplicate data

In [16]:
data = pd.read_csv('data_droped_nov10.csv')

## Drop data that has same values accross entire timestep

In [ ]:
for key, _ in data.carpark_number.value_counts().items():
    if data[(data.carpark_number == key)].lots_available.value_counts().max() == data[(data.carpark_number == key)].shape[0]:
        data.drop(data[data.carpark_number == key].index, inplace=True)

## Save 2nd cleaned version

In [18]:
data.to_csv('data_droped_nov11.csv',sep=',')

## 3rd cleaning cycle started as found duplicate data

In [19]:
data = pd.read_csv('data_droped_nov11.csv')

## Drop data with higher than single total lot value

In [20]:
for key, _ in data.carpark_number.value_counts().items():
    if len(data[data.carpark_number == key].total_lot.unique()) >= 2:
            value = data[data.carpark_number == key].total_lot.value_counts().keys()[0]
            data[data.carpark_number == key] = data[data.carpark_number == key].assign(total_lot = value)

## Drop data with higher than single lot type value

In [21]:
for key, _ in data.carpark_number.value_counts().items():
    if len(data[data.carpark_number == key].lot_type.unique()) >= 2:
            value = data[data.carpark_number == key].lot_type.value_counts().keys()[0]
            data[data.carpark_number == key] = data[data.carpark_number == key].assign(lot_type = value)

In [24]:
data['diff'] = data['total_lot'] - data['lots_available']

In [25]:
data[data['diff'] <0].carpark_number.value_counts()

BJ49    957
DWST    892
SE41    155
SB17    118
S36L     54
SDM2      1
SD2       1
Name: carpark_number, dtype: int64

## Drop data which number of available parking lots is higher than total lot number

In [26]:
data.drop(data[data.carpark_number == 'BJ49'].index, inplace=True)
data.drop(data[data.carpark_number == 'SE41'].index, inplace=True)

## Manual replacement outlier values based on EDA

In [27]:
data.loc[687855,'lots_available'] = 0
data.loc[687856,'lots_available'] = 38

## Save 3rd cleaned version

In [28]:
data.to_csv('data_droped_nov18.csv',sep=',')

## Merge time-series data with other feature dataset related to corresponding parking lots

In [4]:
data = pd.read_csv('data_droped_nov18.csv')

In [5]:
integrated_df= pd.merge(data,D2, how='left', left_on='carpark_number', right_on='car_park_no')

## Convert categorical variables to dummy variable for model training

In [6]:
cat_cols = [ "car_park_type",'short_term_parking','free_parking','night_parking','car_park_basement']

In [7]:
pd.get_dummies(integrated_df['car_park_type']) 
pd.get_dummies(integrated_df['short_term_parking'])
pd.get_dummies(integrated_df['free_parking'])
pd.get_dummies(integrated_df['night_parking'])
pd.get_dummies(integrated_df['car_park_basement'])

,N,Y
0,1,0
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
3868363,1,0
3868364,1,0
3868365,1,0
3868366,1,0


In [8]:
for i in cat_cols:
    dummy = pd.get_dummies(integrated_df[i])
    integrated_df = pd.concat([integrated_df,dummy], axis=1)

In [9]:
integrated_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,total_lot,lot_type,lots_available,carpark_number,datetime,diff,car_park_no,...,7AM-7PM,NO,WHOLE DAY,NO,SUN & PH FR 1PM-10.30PM,SUN & PH FR 7AM-10.30PM,NO,YES,N,Y
0,0,0,0,105,C,18,HE12,2022-09-22 20:45:00,87,HE12,...,0,0,1,0,0,1,0,1,1,0
1,1,1,1,583,C,288,HLM,2022-09-22 20:45:00,295,HLM,...,0,0,1,1,0,0,0,1,1,0
2,2,2,2,329,C,154,RHM,2022-09-22 20:45:00,175,RHM,...,0,0,1,0,0,1,0,1,1,0
3,3,3,3,97,C,61,BM29,2022-09-22 20:45:00,36,BM29,...,0,0,1,1,0,0,0,1,0,1
4,4,4,4,96,C,52,Q81,2022-09-22 20:45:00,44,Q81,...,0,0,1,1,0,0,0,1,1,0


## Drop unnecessary columns

In [10]:
cat_cols = ['Unnamed: 0','lot_type','car_park_no','address','type_of_parking_system', "car_park_type",'short_term_parking','free_parking','night_parking','car_park_basement']

In [11]:
integrated_df = integrated_df.drop(cat_cols, axis=1)

## Save final version

In [ ]:
for index, key in enumerate(integrated_df.carpark_number.value_counts().keys()):
    # print(index, key)
    integrated_df.replace(key, index, inplace=True)

In [13]:
integrated_df.to_csv('data_droped_nov18_dummy_final.csv', sep=',')